In [1]:
# Example 16.2 One-Way Panel Data Analysis, Deviation Approach
# Production of Airline Services: C = f(Q,PF,LF)
# Panel data: 6 airline companies, 15 years (1970-1984)
# Fixed effects and random effects models

In [4]:
import numpy as np
import pandas as pd
from scipy import stats
# set up panel data
pdata = pd.read_csv("http://web.pdx.edu/~crkl/ceR/data/airline.txt", sep='\s+', nrows=90, index_col=['I','T'])
pdata
# alternatively
# data = pd.read_csv("http://web.pdx.edu/~crkl/ceR/data/airline.txt", sep='\s+', nrows=90)
# Set data as panel data
# pdata = data.set_index(['I', 'T'], inplace=True)

C         Q       PF        LF
I T                                       
1 1   1140640  0.952757   106650  0.534487
  2   1215690  0.986757   110307  0.532328
  3   1309570  1.091980   110574  0.547736
  4   1511530  1.175780   121974  0.540846
  5   1676730  1.160170   196606  0.591167
...       ...       ...      ...       ...
6 11   381478  0.112640   874818  0.517766
  12   506969  0.154154  1013170  0.580049
  13   633388  0.186461   930477  0.556024
  14   804388  0.246847   851676  0.537791
  15  1009500  0.304013   819476  0.525775

[90 rows x 4 columns]

In [5]:
pdata.describe()

,C,Q,PF,LF
count,9.000000e+01,90.000000,9.000000e+01,90.000000
mean,1.122524e+06,0.544995,4.716830e+05,0.560460
std,1.192075e+06,0.533586,3.295029e+05,0.052793
min,6.897800e+04,0.037682,1.037950e+05,0.432066
25%,2.920460e+05,0.142128,1.298475e+05,0.528806
50%,6.370010e+05,0.305028,3.574335e+05,0.566085
75%,1.345968e+06,0.945278,8.498398e+05,0.594658
max,4.748320e+06,1.936460,1.015610e+06,0.676287


In [6]:
# variable transfermation
cs = np.log(pdata.C)
qs = np.log(pdata.Q)
pfs = np.log(pdata.PF)
lfs = pdata.LF # load factor, not logged
df = pd.DataFrame({"cs": cs, "qs": qs, "pfs": pfs, "lfs": lfs})
# Descriptive statistics
df.describe()

,cs,qs,pfs,lfs
count,90.000000,90.000000,90.000000,90.000000
mean,13.365609,-1.174309,12.770359,0.560460
std,1.131971,1.150606,0.812375,0.052793
min,11.141543,-3.278573,11.550173,0.432066
25%,12.584100,-1.951053,11.774096,0.528806
50%,13.364510,-1.187357,12.786697,0.566085
75%,14.112500,-0.056371,13.652803,0.594658
max,15.373301,0.660862,13.831000,0.676287


In [7]:
# Pooled OLS estimator
from linearmodels import PooledOLS
pooled = PooledOLS.from_formula('cs ~ 1 + qs + pfs + lfs', df).fit()
print(pooled)

                          PooledOLS Estimation Summary                          
Dep. Variable:                     cs   R-squared:                        0.9883
Estimator:                  PooledOLS   R-squared (Between):              0.9866
No. Observations:                  90   R-squared (Within):               0.9914
Date:                Sat, Apr 16 2022   R-squared (Overall):              0.9883
Time:                        12:21:53   Log-likelihood                    61.770
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2419.3
Entities:                           6   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                    F(3,86)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             2419.3
                            

In [8]:
# Between estimator
from linearmodels import BetweenOLS
between = BetweenOLS.from_formula('cs ~ 1 + qs + pfs + lfs', df).fit()
print(between)


                         BetweenOLS Estimation Summary                          
Dep. Variable:                     cs   R-squared:                        0.9936
Estimator:                 BetweenOLS   R-squared (Between):              0.9936
No. Observations:                   6   R-squared (Within):              -53.198
Date:                Sat, Apr 16 2022   R-squared (Overall):             -17.710
Time:                        12:22:14   Log-likelihood                    7.2182
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      104.12
Entities:                           6   P-value                           0.0095
Avg Obs:                       15.000   Distribution:                     F(3,2)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             104.12
                            

In [9]:
# First differences estimator (without constant term)
from linearmodels import FirstDifferenceOLS
firstdiff = FirstDifferenceOLS.from_formula('cs ~ qs + pfs + lfs', df).fit()
print(firstdiff)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                     cs   R-squared:                        0.9180
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.3613
No. Observations:                  84   R-squared (Within):               0.9854
Date:                Sat, Apr 16 2022   R-squared (Overall):              0.3628
Time:                        12:22:34   Log-likelihood                    140.24
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      302.12
Entities:                           6   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                    F(3,81)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             302.12
                            

In [10]:
from linearmodels.panel.results import compare
res1 = {'Pooled':pooled,'Between':between,'firstdiff':firstdiff}
print(compare(res1))

                          Model Comparison                          
                            Pooled      Between            firstdiff
--------------------------------------------------------------------
Dep. Variable                   cs           cs                   cs
Estimator                PooledOLS   BetweenOLS   FirstDifferenceOLS
No. Observations                90            6                   84
Cov. Est.               Unadjusted   Unadjusted           Unadjusted
R-squared                   0.9883       0.9936               0.9180
R-Squared (Within)          0.9914      -53.198               0.9854
R-Squared (Between)         0.9866       0.9936               0.3613
R-Squared (Overall)         0.9883      -17.710               0.3628
F-statistic                 2419.3       104.12               302.12
P-value (F-stat)            0.0000       0.0095               0.0000
===================== ============ ============ ====================
Intercept                   9.5169

In [11]:
# Fixed effects or within estimator
# with constant inclued or not, will have the same results
# with constant term surpressed
from linearmodels import PanelOLS
fixed = PanelOLS.from_formula('cs ~ qs + pfs + lfs + EntityEffects', df).fit()
print(fixed)
# extract fixed effects
fixed.estimated_effects
fixed_effects = fixed.estimated_effects.unstack(level=0).values[0]
print(fixed_effects)
# F test for fixed effects versus OLS
print(fixed.f_pooled)

                          PanelOLS Estimation Summary                           
Dep. Variable:                     cs   R-squared:                        0.9926
Estimator:                   PanelOLS   R-squared (Between):              0.4742
No. Observations:                  90   R-squared (Within):               0.9926
Date:                Sat, Apr 16 2022   R-squared (Overall):              0.4754
Time:                        12:23:11   Log-likelihood                    130.09
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3604.8
Entities:                           6   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                    F(3,81)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             3604.8
                            

In [12]:
# with constant term included 
fixed1 = PanelOLS.from_formula('cs ~ 1 + qs + pfs + lfs + EntityEffects', df).fit()
print(fixed1)
# extract fixed effects
fixed1.estimated_effects
fixed1_effects = fixed1.params.Intercept + fixed1.estimated_effects.unstack(level=0).values[0]
print(fixed1_effects)

                          PanelOLS Estimation Summary                           
Dep. Variable:                     cs   R-squared:                        0.9926
Estimator:                   PanelOLS   R-squared (Between):              0.9825
No. Observations:                  90   R-squared (Within):               0.9926
Date:                Sat, Apr 16 2022   R-squared (Overall):              0.9860
Time:                        12:23:39   Log-likelihood                    130.09
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3604.8
Entities:                           6   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                    F(3,81)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             3604.8
                            

In [13]:
# F test for fixed effects versus OLS
print(fixed1.f_pooled)

Pooled F-statistic
H0: Effects are zero
Statistic: 57.7321
P-value: 0.0000
Distributed: F(5,81)


In [14]:
# Random effects estimator, constant term must be included
# should not have EntityEffects or TimeEffects in the formula
from linearmodels import RandomEffects
random = RandomEffects.from_formula('cs ~ 1 + qs + pfs + lfs', df).fit()
print(random)
# extract fixed effects
random.estimated_effects
random_effects = random.params.Intercept + random.estimated_effects.unstack(level=0).values[0]
print(random_effects)
print(random.variance_decomposition)

                        RandomEffects Estimation Summary                        
Dep. Variable:                     cs   R-squared:                        0.9923
Estimator:              RandomEffects   R-squared (Between):              0.9839
No. Observations:                  90   R-squared (Within):               0.9925
Date:                Sat, Apr 16 2022   R-squared (Overall):              0.9869
Time:                        12:24:13   Log-likelihood                    127.26
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3697.1
Entities:                           6   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                    F(3,86)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             3697.1
                            

In [15]:
# compare fixed effects and random effects models
res2 = {'Pooled':pooled,'Fixed+1':fixed1,'Fixed':fixed,'Random':random}
print(compare(res2))

effects = pd.DataFrame({'Fixed Effects':fixed_effects,'Random Effects':random_effects},
                       index=pdata.index.levels[0])
print(effects)

                                   Model Comparison                                   
                                Pooled        Fixed+1          Fixed            Random
--------------------------------------------------------------------------------------
Dep. Variable                       cs             cs             cs                cs
Estimator                    PooledOLS       PanelOLS       PanelOLS     RandomEffects
No. Observations                    90             90             90                90
Cov. Est.                   Unadjusted     Unadjusted     Unadjusted        Unadjusted
R-squared                       0.9883         0.9926         0.9926            0.9923
R-Squared (Within)              0.9914         0.9926         0.9926            0.9925
R-Squared (Between)             0.9866         0.9825         0.4742            0.9839
R-Squared (Overall)             0.9883         0.9860         0.4754            0.9869
F-statistic                     2419.3     

In [16]:
# LM test for random effects versus OLS
n = pdata.index.levels[0].size
T = pdata.index.levels[1].size
D = np.kron(np.eye(n), np.ones(T)).T
e = pooled.resids
LM = (e.dot(D).dot(D.T).dot(e) / e.dot(e) - 1) ** 2 * n * T / 2 / (T - 1)
LM_pvalue = stats.chi2(1).sf(LM)
print("LM Test: chisq = {0}, df = 1, p-value = {1}".format(LM, LM_pvalue))

LM Test: chisq = 334.8503622229872, df = 1, p-value = 8.441020390904416e-75


In [17]:
# Hausman test for fixed versus random effects model
# null hypothesis: random effects model
psi = fixed.cov - random.cov.iloc[1:,1:]
diff = fixed.params - random.params[1:]
# psi = fixed1.cov.iloc[1:,1:] - random.cov.iloc[1:,1:]
# diff = fixed1.params[1:] - random.params[1:]
W = diff.dot(np.linalg.inv(psi)).dot(diff)
dof = random.params.size -1
pvalue = stats.chi2(dof).sf(W)
print("Hausman Test: chisq = {0}, df = {1}, p-value = {2}".format(W, dof, pvalue))


Hausman Test: chisq = 2.124706443726442, df = 3, p-value = 0.5469306749586593


In [18]:
#Wald Test

# alternative hausman test based on random effects model
# include group means in the random effects model
# test the significance of group mean coefficients
df2 = df
df2['qsm'] = np.kron(df2.qs.mean(level=0), np.ones(T))
df2['pfsm'] = np.kron(df2.pfs.mean(level=0), np.ones(T))
df2['lfsm'] = np.kron(df2.lfs.mean(level=0), np.ones(T))
random1 = RandomEffects.from_formula('cs ~ 1 + qs + pfs + lfs + qsm + pfsm + lfsm', df2).fit(cov_type='clustered',cluster_entity=True)
print(random1)
# hypothsis testing of 'qsm=pfsm=lfsm=0'
print(random1.wald_test(formula='qsm=pfsm=lfsm=0'))



C:\Users\HP\AppData\Local\Temp/ipykernel_2776/3774763024.py:7: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  df2['qsm'] = np.kron(df2.qs.mean(level=0), np.ones(T))
C:\Users\HP\AppData\Local\Temp/ipykernel_2776/3774763024.py:8: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  df2['pfsm'] = np.kron(df2.pfs.mean(level=0), np.ones(T))
C:\Users\HP\AppData\Local\Temp/ipykernel_2776/3774763024.py:9: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  df2['lfsm'] = np.kron(df2.lfs.mean(level=0), np.ones(T))


                        RandomEffects Estimation Summary                        
Dep. Variable:                     cs   R-squared:                        0.9933
Estimator:              RandomEffects   R-squared (Between):              0.9936
No. Observations:                  90   R-squared (Within):               0.9926
Date:                Sat, Apr 16 2022   R-squared (Overall):              0.9933
Time:                        12:26:44   Log-likelihood                    86.679
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2040.9
Entities:                           6   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                    F(6,83)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):          2.072e+12
                            

In [19]:
# panel robust hetero cov
fixed_robust = PanelOLS.from_formula('cs ~ 1 + qs + pfs + lfs + EntityEffects', df).fit(cov_type='clustered',cluster_entity=True)
print(fixed_robust)
random_robust = RandomEffects.from_formula('cs ~ 1 + qs + pfs + lfs', df).fit(cov_type='clustered',cluster_entity=True)
print(random_robust)

                          PanelOLS Estimation Summary                           
Dep. Variable:                     cs   R-squared:                        0.9926
Estimator:                   PanelOLS   R-squared (Between):              0.9825
No. Observations:                  90   R-squared (Within):               0.9926
Date:                Sat, Apr 16 2022   R-squared (Overall):              0.9860
Time:                        12:27:10   Log-likelihood                    130.09
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3604.8
Entities:                           6   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                    F(3,81)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             768.64
                            

In [20]:
# compare fixed effects and random effects models
res3 = {'Fixed (Panel-Robust)':fixed_robust,'Random (Panel-Robust)':random_robust}
print(compare(res3))

                         Model Comparison                         
                        Fixed (Panel-Robust) Random (Panel-Robust)
------------------------------------------------------------------
Dep. Variable                             cs                    cs
Estimator                           PanelOLS         RandomEffects
No. Observations                          90                    90
Cov. Est.                          Clustered             Clustered
R-squared                             0.9926                0.9923
R-Squared (Within)                    0.9926                0.9925
R-Squared (Between)                   0.9825                0.9839
R-Squared (Overall)                   0.9860                0.9869
F-statistic                           3604.8                3697.1
P-value (F-stat)                      0.0000                0.0000
=====================            ===========       ===============
Intercept                             9.7135                9.